In [1]:
import json
import pandas as pd
from tqdm.notebook import tqdm

from variablesOtherPrior import *

#### Read in the event file, weights and parts of the regression dataset

In [2]:
# Event file.
event_file = pd.read_csv('../data/final_eventfile_buckeye.gz', 
                         sep = '\t', 
                         low_memory = True,
                         engine = 'c')

# The word column from the regression dataframe.  
speaker_word = pd.read_csv('../data/regression_data.csv',
                           usecols=['wordID'],
                           dtype={"wordID": "category"},
                           low_memory = True, 
                           engine = 'c')

# Weights. 
df = xr.open_dataarray('../output/weights/weights_buckeye.nc')
weight_matrix = df.to_pandas()
weight_matrix = weight_matrix.transpose()
weight_matrix.info(verbose=False, memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
Index: 13809 entries, c.okay to y.jhihts
Columns: 9558 entries, okay to looser
dtypes: float64(9558)
memory usage: 1007.8 MB


#### Add prior to the dataframe

In [3]:
words = speaker_word['wordID'].tolist()

prior_dict = {}

# Make a prior dictionary.
for index, word in tqdm(enumerate(words)):
    if word not in prior_dict.keys(): 

        prior_all = get_prior(weight_matrix = weight_matrix, word_outcome = word, domain_specific = False)
        priors = get_prior(weight_matrix = weight_matrix, word_outcome = word, domain_specific = True)
        
        prior_dict[word] = {'prior_all': prior_all, 
                            'prior_segments': priors['Segment'], 
                            'prior_syllables': priors['Syllable'], 
                            'prior_context' : priors['Context']} 
# Save to json. 
out_file = open("../data/otherPrior_dictionary.json", "w")
json.dump(prior_dict, out_file, indent = 6)
out_file.close()
    
df = pd.DataFrame({'prior_all': [], 'prior_segments': [], 'prior_syllables' : [], 'prior_context': []})

print('Filling..')
for index, word in tqdm(enumerate(words)):
    df.at[index, 'prior_all'] = prior_dict[word]['prior_all']
    df.at[index, 'prior_segments'] = prior_dict[word]['prior_segments']
    df.at[index, 'prior_syllables'] = prior_dict[word]['prior_syllables']
    df.at[index, 'prior_context'] = prior_dict[word]['prior_context']

print('Loading df...')
# Loading whole regression dataset.
regression_data = pd.read_csv('../data/regression_data.csv', 
                              dtype={"speakerID": "category","speakerAge": "category", "speakerGender": "category",
                                     "interviewerGender": "category", "wordID": "category", "wordDur": "float",
                                     "wordPOS": "category", "n_segments": "category", "n_syllables": "category"}, 
                              engine = 'c',
                              low_memory = True)
print('Concating...')
result = pd.concat(objs = [regression_data, df], axis = 1)
result.to_csv('../data/regression_data_otherPrior.csv', index = False)

276845it [14:40, 314.25it/s] 


Filling..


276845it [45:38, 101.10it/s]


Loading df...
Concating...


TypeError: concat() got an unexpected keyword argument 'objects'

#### Add activation to the dataframe

In [ ]:
words = speaker_word['wordID'].tolist()

df = pd.DataFrame({'activation_all': [], 'activation_segments': [], 'activation_syllables' : [], 
                   'activation_context': []})

for index, word in tqdm(enumerate(words)): 
    if index == 0:
        c1 = 'c.' + words[index+1]
        c2 = None
    elif index == len(words)-1:
        c1 = 'c.' + words[index-1]
        c2 = None
    else:
        c1 = 'c.' + words[index-1]
        c2 = 'c.' + words[index+1]
    
    act = activation(word_outcome = word, c1 = c1, c2=c2, 
           event_files = [event_file], weight_matrix = weight_matrix, 
           domain_specific = False)
    act_domain = activation(word_outcome = word, c1 = c1, c2=c2,  
           event_files = [event_file], weight_matrix = weight_matrix, 
           domain_specific = True)
    
    df.at[index, 'activation_all'] = act
    df.at[index, 'activation_segments'] = act_domain['Segment']
    df.at[index, 'activation_syllables'] = act_domain['Syllable']
    df.at[index, 'activation_context'] = act_domain['Context']
    
print('Loading data')
# Loading whole regression dataset.
regression_data = pd.read_csv('../data/regression_data_prior.csv', 
                              dtype={"speakerID": "category","speakerAge": "category", "speakerGender": "category",
                                     "interviewerGender": "category", "wordID": "category", "wordDur": "float",
                                     "wordPOS": "category", "n_segments": "category", "n_syllables": "category"}, 
                              engine = 'c',
                              low_memory = True)    

ende = pd.concat(obs = [regression_data, df], axis = 1)
ende.to_csv('../data/regression_data_activation.csv', index = False)

In [ ]:
ende = pd.concat(objs = [regression_data, df], axis = 1)

In [ ]:
ende.to_csv('../data/regression_data_activation.csv', index = False)